In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import requests
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
# Load the data
df = pd.read_csv('TransformGainesville_CrimesALL.csv', index_col = 0)

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'TransformGainesville_CrimesALL.csv'

In [ ]:
# Change name of column "offenseDate" to date
df.rename(columns={'offenseDate': 'Date'}, inplace=True)
df.head()

In [ ]:
df['Date'] = pd.to_datetime(df.Date, format='%Y-%m-%d')

In [ ]:
df['Date'] = pd.to_datetime(df['Date']).dt.date


In [ ]:
url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Gainesville,FL/2018-01-01/2021-12-31?unitGroup=us&key=JVFDPCT4LWWPVKADN783XGRVA&include=days&elements=datetime,moonphase'

In [ ]:
r = requests.get(url)

In [ ]:
json = r.json()
json

In [ ]:
json.keys()

In [ ]:
moonphases_df = pd.DataFrame(json['days'])
moonphases_df

In [ ]:
json['days']

In [ ]:
moonphases_df.dtypes

In [ ]:
bins = [.0 , .25 , .50 , .75, 1 ]

In [ ]:
moonphases_df['moonPhases_cat'] = pd.cut(moonphases_df['moonphase'], bins)



In [ ]:
moonphases_df

In [ ]:
# Labels = 1 = New Moon , 2 = Full Moon, 3 Third Quarter =  4 = back to new moon

labels =[1,2,3,4]

moonphases_df['moonPhases'] = pd.cut(moonphases_df['moonphase'], bins,labels=labels)
print (moonphases_df)


In [ ]:
moonphases_df

In [ ]:
new_moonphasesdf = moonphases_df[['datetime','moonPhases']]
new_moonphasesdf

In [ ]:
new_moonphasesdf.rename(columns={'datetime': 'Date'}, inplace=True)
new_moonphasesdf.head()

In [ ]:
new_moonphasesdf['Date'] = pd.to_datetime(new_moonphasesdf.Date, format='%Y-%m-%d')


In [ ]:
new_moonphasesdf['Date'] = pd.to_datetime(new_moonphasesdf['Date']).dt.date

In [ ]:
new_moonphasesdf

In [ ]:
df.columns

In [ ]:
#Find out unique names of CFS, CFS_type, and moonPhase

In [ ]:
df.Classification.unique()

In [ ]:
df.CFS.unique()

In [ ]:
df.CFS_Type.unique()

In [ ]:
df.moonPhase.unique()

In [ ]:
# Assign the uncessary columns to variable and drop

unused_cols = ['CFS','reportDate', 'offenseHour','reportHour','reportDOW','city','state','address','longitude','latitude','day','location','date ','month','year', 'moonPhase']

df = df.drop(unused_cols, axis = 1)

df.head(50)

In [ ]:
# Gather CFS_Type and moonPhase columns for get dummies

convert_cols = df["CFS_Type"]
convert_cols

In [ ]:
# Convert CFS_Type and moonPhase with get dummies in dataframe

new_columns_df = pd.get_dummies(df['CFS_Type'])
new_columns_df

In [ ]:
# Go back and take out the original CFS_Type columns and moonPhase  in the original df

df = df.drop(['CFS_Type'], axis = 1)


In [ ]:
new_df = pd.merge(df,new_columns_df, left_index=True, right_index = True )
new_df

In [ ]:
# Join the new_moonphasesdf to df on datetime 

dfcomplete = pd.merge(new_df,new_moonphasesdf, left_on = "Date", right_on = 'Date')
dfcomplete.head

In [ ]:
y = pd.get_dummies(new_df['Classification'])


X = dfcomplete[['Battery', 'Quality of Life', 'Theft', 'Other', 'Alcohol',
       'Death Inv/Homicide', 'Gov Reg Vio', 'Drugs', 'Fraud', 'Assault',
       'Suicide','moonPhases']]

In [ ]:
y.value_counts()

In [ ]:
X.describe()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# You can modify test_size to account for 30% testing/70% training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5)

In [ ]:
# Resample the training data with the RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
# https://scikit-learn.org/stable/modules/multiclass.html

In [ ]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
#Confusion Matrix

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1))
print(cm)

In [ ]:
# Print the  classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
# MODULE DIRECTIONS FOR CONFUSION MATRIX

# Display the confusion matrix
#cm = confusion_matrix(y_test, y_pred)
#cm_df = pd.DataFrame(
    #cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
#cm_df

In [ ]:
# We need data with every single moon phase attached to each event to have a better data set. The outputs from the classification report tell you that because its getting 1.00 precision on 3/4 data points because theres only 8 moon phases for 40k events.  